In [1]:
import pandas as pd
import numpy as np
from collections import Counter

# Visualization
import scattertext as st
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Preprocessing Tools
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import spacy

# Document-Term Matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Topic Modeling
from sklearn.decomposition import TruncatedSVD, NMF

# Topic Modeling
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

# Load Data

In [2]:
# Read in the pickle file
df = pd.read_pickle('df_sentiment_score_spacy_docs_cleaned_pkl')

# Divide Reviews into Subsets

In [3]:
# Set the display constraints to be able to read entire reviews
pd.set_option('display.max_colwidth', None)

df.sample(3)

,ID,Rating,Date,Reviewer_Location,Reviews,Branch,Rating_Type,Sentiment_Score,spacy_doc,docs_clean
30511,575146297,5,2018-3,United States,"Such a fun day with my mother and sister! Fun memories together that we never got to make when we were younger, but now we've got some & they will last us a lifetime!",Paris,Positive,0.5972,"(Such, a, fun, day, with, my, mother, and, sister, !, Fun, memories, together, that, we, never, got, to, make, when, we, were, younger, ,, but, now, we, 've, got, some, &, they, will, last, us, a, lifetime, !)","[fun, day, mother, sister, fun, memory, get, young, get, lifetime]"
30998,547397695,5,2017-11,United Kingdom,"Spent 4 days here with friends last month and had yet another amazing visit. The new parades are just as amazing as ever and the illuminations on the castle was fantastic, would recommend seeing. Loved all the rides especially Thunder Mountain in the dark. The single rider queues were a good idea and used it a couple of times. Would recommend booking restaurants in the park before arriving if possible, the ones we wanted to go to were all fully booked. All staff we so friendly and helpful. Cant wait to return again one day.",Paris,Positive,0.9864,"(Spent, 4, days, here, with, friends, last, month, and, had, yet, another, amazing, visit, ., The, new, parades, are, just, as, amazing, as, ever, and, the, illuminations, on, the, castle, was, fantastic, ,, would, recommend, seeing, ., Loved, all, the, rides, especially, Thunder, Mountain, in, the, dark, ., The, single, rider, queues, were, a, good, idea, and, used, it, a, couple, of, times, ., Would, recommend, booking, restaurants, in, the, park, before, arriving, if, possible, ,, the, ones, we, wanted, to, go, to, were, all, fully, booked, ., All, staff, we, so, friendly, and, helpful, ., Ca, nt, wait, to, ...)","[spend, day, friend, month, amazing, visit, new, parade, amazing, illumination, castle, fantastic, recommend, seeing, love, ride, especially, thunder, mountain, dark, single, rider, queue, good, idea, couple, time, recommend, book, restaurant, park, arrive, possible, one, want, fully, book, staff, friendly, helpful, nt, wait, return, day]"
32099,481408468,4,2017-4,Norway,"Visited Disneyland Park with a friend. We had purchased tickets online, so it didn't take that long to get in. The atmosphere in the park is nice. Unfortunately, it seemed like we visited on a busy day, so the lines were long. Getting fast passes for the rides you really want to ride is strongly recommended. There is also an app that shows wait times for the different rides, and this can be quite helpful for planning where to go.",Paris,Positive,0.9133,"(Visited, Disneyland, Park, with, a, friend, ., We, had, purchased, tickets, online, ,, so, it, did, n't, take, that, long, to, get, in, ., The, atmosphere, in, the, park, is, nice, ., Unfortunately, ,, it, seemed, like, we, visited, on, a, busy, day, ,, so, the, lines, were, long, ., Getting, fast, passes, for, the, rides, you, really, want, to, ride, is, strongly, recommended, ., There, is, also, an, app, that, shows, wait, times, for, the, different, rides, ,, and, this, can, be, quite, helpful, for, planning, where, to, go, .)","[visited, disneyland, park, friend, purchase, ticket, online, long, atmosphere, park, nice, unfortunately, like, visit, busy, day, line, long, get, fast, pass, ride, want, ride, strongly, recommend, app, show, wait, time, different, ride, helpful, plan]"


In [4]:
# Disneyland - California
calif = df[df['Branch']=='California']
calif.shape

(17745, 10)

In [5]:
# Disneyland - Paris
paris = df[df['Branch']=='Paris']
paris.shape

(11547, 10)

In [6]:
# Disneyland - HongKong
hkong = df[df['Branch']=='HongKong']
hkong.shape

(8255, 10)

In [36]:
hkong.sample(2)

,ID,Rating,Date,Reviewer_Location,Reviews,Branch,Rating_Type,Sentiment_Score,spacy_doc,docs_clean
1553,547367670,4,2017-12,Malaysia,"parks are well maintained. Staff are friendly. In comparison with Tokyo disneyland, this park is relatively small and have less games. if you plan properly, you can play most of the games within a day. The toys in the shops are expensive.",HongKong,Positive,0.7717,"(parks, are, well, maintained, ., Staff, are, friendly, ., , In, comparison, with, Tokyo, disneyland, ,, this, park, is, relatively, small, and, have, less, games, ., , if, you, plan, properly, ,, you, can, play, most, of, the, games, within, a, day, ., , The, toys, in, the, shops, are, expensive, .)","[park, maintain, staff, friendly, , comparison, tokyo, disneyland, park, relatively, small, game, , plan, properly, play, game, day, , toy, shop, expensive]"
3492,415568547,2,2016-9,Malaysia,"Part of it why I didnt enjoyed was because it was raining the day I went. And I found out they charged for the raincoat at HKD 65, and some of their rides are even closed.",HongKong,Negative,-0.5426,"(Part, of, it, why, I, did, nt, enjoyed, was, because, it, was, raining, the, day, I, went, ., And, I, found, out, they, charged, for, the, raincoat, at, HKD, 65, ,, and, some, of, their, rides, are, even, closed, .)","[not, enjoy, rain, day, go, find, charge, raincoat, hkd, ride, closed]"


# POS by Positive Rating

In [9]:
# Filter out the positive reviews
calif_pos_reviews = calif[calif['Rating_Type']=='Positive']

# Extract the adjectives in the positive reviews
calif_pos_adj = [token.text.lower() for doc in calif_pos_reviews.spacy_doc for token in doc if token.pos_=='ADJ']

# Find the 20 most common adjectives in the positive reviews
calif_top_pos_adj = Counter(calif_pos_adj).most_common(20)
calif_top_pos_adj

[('great', 6936),
 ('fast', 3550),
 ('good', 3361),
 ('many', 3044),
 ('first', 3037),
 ('more', 3021),
 ('long', 2723),
 ('best', 2514),
 ('old', 2407),
 ('other', 2374),
 ('worth', 2288),
 ('little', 2278),
 ('amazing', 2259),
 ('most', 1805),
 ('much', 1758),
 ('sure', 1758),
 ('magical', 1723),
 ('few', 1685),
 ('fun', 1645),
 ('busy', 1644)]

In [10]:
# Filter out the positive reviews
paris_pos_reviews = paris[paris['Rating_Type']=='Positive']

# Extract the adjectives in the positive reviews
paris_pos_adj = [token.text.lower() for doc in paris_pos_reviews.spacy_doc for token in doc if token.pos_=='ADJ']

# Find the 20 most common adjectives in the positive reviews
paris_top_pos_adj = Counter(paris_pos_adj).most_common(20)
paris_top_pos_adj

[('great', 4725),
 ('good', 4217),
 ('amazing', 2475),
 ('worth', 2248),
 ('fast', 2214),
 ('expensive', 2177),
 ('more', 2172),
 ('little', 2031),
 ('old', 1949),
 ('other', 1931),
 ('long', 1926),
 ('magical', 1897),
 ('many', 1879),
 ('best', 1703),
 ('few', 1679),
 ('most', 1614),
 ('first', 1572),
 ('busy', 1504),
 ('big', 1387),
 ('fantastic', 1364)]

In [11]:
# Filter out the positive reviews
hkong_pos_reviews = hkong[hkong['Rating_Type']=='Positive']

# Extract the adjectives in the positive reviews
hkong_pos_adj = [token.text.lower() for doc in hkong_pos_reviews.spacy_doc for token in doc if token.pos_=='ADJ']

# Find the 20 most common adjectives in the positive reviews
hkong_top_pos_adj = Counter(hkong_pos_adj).most_common(20)
hkong_top_pos_adj

[('great', 2741),
 ('good', 2320),
 ('small', 1589),
 ('other', 1517),
 ('more', 1377),
 ('many', 1182),
 ('worth', 1108),
 ('long', 1088),
 ('best', 1055),
 ('most', 1027),
 ('first', 947),
 ('old', 944),
 ('nice', 927),
 ('smaller', 897),
 ('expensive', 844),
 ('big', 817),
 ('little', 807),
 ('amazing', 801),
 ('fun', 764),
 ('easy', 758)]

# POS by Negative Rating

In [12]:
# Do the same for the negative reviews
calif_neg_reviews = calif[calif['Rating_Type']=='Negative']

calif_neg_adj = [token.text.lower() for doc in calif_neg_reviews.spacy_doc for token in doc if token.pos_=='ADJ']

# Top 20 common adjectives in negative reviews
calif_neg_adj = Counter(calif_neg_adj).most_common(20)
calif_neg_adj

[('many', 530),
 ('long', 379),
 ('fast', 376),
 ('other', 340),
 ('more', 338),
 ('good', 284),
 ('old', 262),
 ('first', 245),
 ('last', 223),
 ('crowded', 217),
 ('great', 207),
 ('few', 206),
 ('expensive', 195),
 ('better', 186),
 ('most', 181),
 ('same', 178),
 ('disappointed', 173),
 ('worth', 171),
 ('much', 161),
 ('small', 158)]

In [13]:
# Do the same for the negative reviews
paris_neg_reviews = paris[paris['Rating_Type']=='Negative']

paris_neg_adj = [token.text.lower() for doc in paris_neg_reviews.spacy_doc for token in doc if token.pos_=='ADJ']

# Top 20 common adjectives in negative reviews
paris_neg_adj = Counter(paris_neg_adj).most_common(20)
paris_neg_adj

[('many', 682),
 ('good', 652),
 ('other', 620),
 ('more', 560),
 ('fast', 514),
 ('long', 493),
 ('expensive', 467),
 ('old', 451),
 ('few', 443),
 ('same', 416),
 ('most', 410),
 ('great', 359),
 ('better', 359),
 ('first', 350),
 ('small', 347),
 ('poor', 344),
 ('open', 322),
 ('little', 308),
 ('rude', 299),
 ('last', 287)]

In [14]:
# Do the same for the negative reviews
hkong_neg_reviews = hkong[hkong['Rating_Type']=='Negative']

hkong_neg_adj = [token.text.lower() for doc in hkong_neg_reviews.spacy_doc for token in doc if token.pos_=='ADJ']

# Top 20 common adjectives in negative reviews
hkong_neg_adj = Counter(hkong_neg_adj).most_common(20)
hkong_neg_adj

[('small', 188),
 ('other', 150),
 ('more', 118),
 ('good', 114),
 ('many', 102),
 ('long', 90),
 ('old', 67),
 ('few', 66),
 ('better', 66),
 ('bad', 64),
 ('worth', 61),
 ('expensive', 61),
 ('little', 58),
 ('much', 55),
 ('rude', 54),
 ('first', 52),
 ('great', 51),
 ('most', 50),
 ('hot', 49),
 ('disappointed', 46)]

# Topic Modeling - LSA with TdifVectorizer

In [22]:
# Merge all tokens together and build Vectorizer with cleaned text data
calif_docs_list_clean = [' '.join(doc) for doc in calif.docs_clean]
paris_docs_list_clean = [' '.join(doc) for doc in paris.docs_clean]
hkong_docs_list_clean = [' '.join(doc) for doc in hkong.docs_clean]

In [19]:
# Function to display top words for each topic
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### Vectorize, Fit the Model

In [18]:
# Disney - California
# Convert documents to document-term matrix using TdifVectorizer, remove English stop words
tfv = TfidfVectorizer(stop_words='english')
X = tfv.fit_transform(calif_docs_list_clean)

df_X = pd.DataFrame(X.toarray(), columns=tfv.get_feature_names_out())

# Fit the model
num_topics=4

calif_lsa_topics = TruncatedSVD(num_topics)
calif_lsa_doc_topic = calif_lsa_topics.fit(X)

In [28]:
# Disney - Paris
# Convert documents to document-term matrix using TdifVectorizer, remove English stop words
tfv = TfidfVectorizer(stop_words='english')
X2 = tfv.fit_transform(paris_docs_list_clean)

df_X2 = pd.DataFrame(X2.toarray(), columns=tfv.get_feature_names_out())

# Fit the model
paris_lsa_topics = TruncatedSVD(num_topics)
paris_lsa_doc_topic = paris_lsa_topics.fit(X2)

In [32]:
# Disney - HongKong
# Convert documents to document-term matrix using TdifVectorizer, remove English stop words
tfv = TfidfVectorizer(stop_words='english')
X3 = tfv.fit_transform(hkong_docs_list_clean)

df_X3 = pd.DataFrame(X3.toarray(), columns=tfv.get_feature_names_out())

# Fit the model
hkong_lsa_topics = TruncatedSVD(num_topics)
hkong_lsa_doc_topic = hkong_lsa_topics.fit(X3)

### Display Topics

In [23]:
# Disney - California
display_topics(calif_lsa_topics, tfv.get_feature_names_out(), 10)


Topic  0
ride, park, time, day, disneyland, disney, great, line, pass, love

Topic  1
place, love, disneyland, happy, earth, disney, year, visit, kid, family

Topic  2
disney, park, world, florida, small, california, adventure, visit, original, walt

Topic  3
day, place, park, earth, happy, pass, hopper, ticket, spend, california


In [30]:
# Disney- Paris
display_topics(paris_lsa_topics, tfv.get_feature_names_out(), 10)


Topic  0
ride, park, day, disneyland, disney, time, visit, kid, good, great

Topic  1
place, kid, enjoy, visit, family, parade, adult, love, firework, fun

Topic  2
kong, hong, disneyland, visit, disney, small, world, compare, paris, place

Topic  3
disney, park, small, great, kid, fun, love, place, child, world


In [33]:
# Disney - Hong Kong
display_topics(hkong_lsa_topics, tfv.get_feature_names_out(), 10)


Topic  0
ride, park, day, disneyland, disney, time, visit, kid, good, great

Topic  1
place, kid, enjoy, visit, family, parade, adult, love, happy, firework

Topic  2
kong, hong, disneyland, visit, disney, small, world, compare, place, paris

Topic  3
disney, park, kid, great, small, place, fun, love, like, world


# Topic Modeling with CorEx 
### Unsupervised, No Anchor Words

In [39]:
# Vectorize the document
vectorizer = CountVectorizer(max_features=2500,
                             stop_words='english', 
                             token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True)

In [41]:
calif_word = vectorizer.fit_transform(calif_docs_list_clean)
calif_words = list(np.asarray(vectorizer.get_feature_names_out()))

paris_word = vectorizer.fit_transform(paris_docs_list_clean)
paris_words = list(np.asarray(vectorizer.get_feature_names_out()))

hkong_word = vectorizer.fit_transform(hkong_docs_list_clean)
hkong_words = list(np.asarray(vectorizer.get_feature_names_out()))

In [50]:
# Disney - California
calif_topic_model = ct.Corex(n_hidden= 5,
                       words=calif_words, 
                       seed=1)

calif_topic_model.fit(calif_word,         
                words= calif_words,     
                docs= calif.docs_clean)

# Topics without anchored words
calif_topics = calif_topic_model.get_topics()
for n,topic in enumerate(calif_topics):
    topic_words,_,_ = zip(*topic)
    print(f'{n}: {", ".join(topic_words)}')

0: people, disney, like, disneyland, way, think, know, tell, thing, try
1: day, park, pass, fast, ticket, early, california, adventure, hotel, use
2: main, parade, street, firework, walk, character, night, mickey, area, sit
3: wait, ride, line, hour, time, long, minute, want, crowd, open
4: mountain, space, jones, indiana, haunted, mansion, thunder, splash, caribbean, star


In [49]:
# Disney - Paris
paris_topic_model = ct.Corex(n_hidden= 5,
                       words=paris_words, 
                       seed=1)

paris_topic_model.fit(paris_word,         
                words= paris_words,     
                docs= paris.docs_clean)

# Topics without anchored words
paris_topics = paris_topic_model.get_topics()
for n,topic in enumerate(paris_topics):
    topic_words,_,_ = zip(*topic)
    print(f'{n}: {", ".join(topic_words)}')

0: hotel, food, stay, queue, eat, day, meal, hour, book, time
1: people, staff, leave, tell, say, way, pay, ask, service, need
2: disney, park, paris, disneyland, like, think, small, minute, look, train
3: character, parade, meet, mickey, castle, princess, little, old, year, night
4: mountain, thunder, space, jones, indiana, peter, buzz, pan, caribbean, coaster


In [51]:
# Disney - Hong Kong
hkong_topic_model = ct.Corex(n_hidden= 5,
                       words=hkong_words, 
                       seed=1)

hkong_topic_model.fit(hkong_word,         
                words= hkong_words,     
                docs= hkong.docs_clean)

# Topics without anchored words
hkong_topics = hkong_topic_model.get_topics()
for n,topic in enumerate(hkong_topics):
    topic_words,_,_ = zip(*topic)
    print(f'{n}: {", ".join(topic_words)}')

0: food, main, street, english, chinese, restaurant, lunch, people, way, good
1: mountain, grizzly, mystic, space, manor, gulch, toy, story, lion, king
2: ticket, wait, buy, long, time, line, minute, day, queue, pass
3: mickey, parade, firework, character, night, water, pm, bring, photo, watch
4: park, disney, hong, kong, small, world, disneyland, think, like, year


# Visualization with Scattertext

In [7]:
# Load our chosen language model
nlp = spacy.load('en_core_web_sm')

In [8]:
# Build corpus ----TAKES TOO LONG TO RUN --- ABANDON!
calif_corpus = st.CorpusFromPandas(
            calif,
            category_col='Rating_Type',
            text_col='Reviews',
            nlp=nlp).build()

In [ ]:
# Create scattertext, save HTML file

calif_html = st.produce_scattertext_explorer(
        calif_corpus,
        category='positive',
        category_name='Positve',
        not_category_name='Negative')